# TME 4 : Projet Detection de motifs

Nous allons développer des algorithmes pour chercher de motifs dans les données de ChipSeq de C. glabrata.
Pour commencer nous allons d'abord produire des données artificielles qui nous permettront de tester rapidement nos algorithmes. Ensuite nous allons chercher les motifs dans C. glabrata et analyser les résultats.

## Partie A : Recherche de pattern (motifs) identiques


1\. Faites une fonction pour générer aléatoirement des séquences artificielles, puis générer t=10 séquences de n=41 nucléotides chacune. Toutes les lettres peuvent être équiprobables pour la génération des séquences.

In [18]:
import random
nuc = ('A', 'C', 'G', 'T')

t=10 #nombre de sequences
n=41 #longueur des sequences

#Pour un simple teste
t=3 #nombre de sequences
n=8 #longueur des sequences

def generateRandomSequence(n, upper=False):
    """
    Genere une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence ="".join([nuc[random.randint(0,3)] for i in range(n)])
    if upper:
        return sequence.lower()
    return sequence

def generateRandomSequences(n, t):
    """
    Genere plusieurs séquences nucléotidiques aléatoires 
    entrée n : longueur des sequences
    entrée t : nombre de sequences
    sortie sequences : liste de sequences nucléotidiques aléatoires 
    """
    return [generateRandomSequence(n,False) for i in range(t)]

seqs = generateRandomSequences(n, t)

print (seqs)


['CCAAGCCT', 'GCTAAGTC', 'AGGTGCCA']


2\. Nous allons maintenant implanter dans les séquences artificielles generés avant un motif de taille `k`=9 à des positions aléatoires (choisies uniformément le long de la séquence). On considère une proportion ``f``=0.9 de séquences qui possèdent le motif.

In [19]:
# taille des motifs k = 9
k = 3 
# frequences d'occurences des motifs f = 0.9 
f = 0.9 

motif=generateRandomSequence(k, False)
#motif='*'*k
motifs=generateRandomSequences(k, t)
import random

def implantMotifs(motifs, f, sequences):
    """
    Insérer un motif dans des positions aléatoires des séquences
    entrée motifs : motif qui va être implanté dans les séquences
    entrée f : fréquence d'implantation
    entrée séquences : liste de sequences
    sortie modified_sequences: liste de séquences ayant parfois le motif implanté
    """
    modified_sequences = []
    for i in range(len(sequences)) :
        if(random.uniform(0,1)<f):
            r1=random.randint(0,len(sequences[i]))
            sequences[i]=sequences[i][:r1]+motifs+sequences[i][r1:]
            r2=random.randint(0,len(sequences[i]))
            sequences[i]=sequences[i][:r2]+motifs+sequences[i][r2:]
            r3=random.randint(0,len(sequences[i]))
            sequences[i]=sequences[i][:r3]+motifs+sequences[i][r3:]
            modified_sequences.append(sequences[i])
    return modified_sequences


print(motifs)
print(seqs)
for i in motifs:
    motif_implanted_seqs =implantMotifs(i, f, seqs)
print(seqs)
print(motif_implanted_seqs)


['CCT', 'CGC', 'TAT']
['CCAAGCCT', 'GCTAAGTC', 'AGGTGCCA']
['CCCCTCCCGCTACGCAGCGCCCTCCT', 'CGTATCCCTGCTACCCTCTCGCAGCGCTCTATATT', 'CCTCCTAGTATGTGCGTTATATCCCGCCTCGCCCA']
['CGTATCCCTGCTACCCTCTCGCAGCGCTCTATATT', 'CCTCCTAGTATGTGCGTTATATCCCGCCTCGCCCA']


3\. Faites une fonction pour chercher les $m$ motifs de taille $k$ les plus fréquents dans l'ensemble des séquences. Tester cette fonction sur un l'ensemble de séquences avec le motif implanté génère dans les questions précédentes. Faite aussi une fonction qu'affiche les $top$ motifs les plus fréquents. 

In [20]:
def searchMotifs(k, sequences):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de sequences
    sortie motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    >>>searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """
    
    motifs  = {}
    for s in sequences:
        for i in range(0,len(s)):
            mot=s[i:i+k]
            if len(mot)==k:
                if mot not in motifs:
                    motifs[mot]=1
                else:
                     motifs[mot]+=1
    return motifs

def getTopMotifs(motifs, top):
    """
    renvoyer les top motifs le plus frequent
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    entrée top : les top plus frequent 
    sortie motifsfreq: dictionnaire contenant les top motifs les plus fréquents, clé=motif, valeur = fréquence d'observation
    >>>getTopMotifs({'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}, 2)
    {'TAA': 3, 'TAT': 2}
    """
    #return dict([(k,v) for k,v in sorted(motifs.items(),key=lambda x: x[1],reverse=True)][:top])
    tmp=[(None,0)]*top #contien le top
    for k in motifs : #nombre de rep egale nombre de motifs
        i=top-1
        while motifs[k] > tmp[i][1] : #nombre de rep boucle while = nombre top recherché
            if motifs[k] > tmp[i-1][1] :
                i-=1
                if i==0:#motif[k] a la plus haute freq parmis tous les motifs
                    tmp[1:]=tmp[0:-1]
                    tmp[i]=(k,motifs[k])
                    break #sortie anticiper : motif k suivant
            else:
                tmp[i+1:]=tmp[i:-1]
                tmp[i]=(k,motifs[k])#motif[k] fait parti du top 
                break #sortie anticiper : motif k suivant
    
    return dict(tmp)
    
    
    #un programme tres performant si le nombre top rechercher est petit.
    #peu prendre un peu plus de temp si le nombre de top est elever.
    #inconvenient implementation dependant du nombre de top, complexiter lineaire.
    #           complexité O(n*top-log(top))
    
    
    #Autre implementation possible pour ce programme  sorted(dict) -> head top:
    
        #def getTopMotifs(motifs, top):
            #return dict([(k,v) for k,v in sorted(motifs.items(),key=lambda x: x[1],reverse=True)][:top])
    
    #avantage et incovenient de cette implementation ça complexité est linéarithmique depend de la methode sort utiliser.
    #A preferer si un nombre de top vraiment consequent est rechercher.
    #          complexité O(n*log(n)+top) en moyenne , O(n²+top) pire cas
                    


print(seqs)
motifsFound = searchMotifs(k, seqs)
print (motifsFound)
topMotifs = getTopMotifs(motifsFound, 4)
print (topMotifs)

['CCCCTCCCGCTACGCAGCGCCCTCCT', 'CGTATCCCTGCTACCCTCTCGCAGCGCTCTATATT', 'CCTCCTAGTATGTGCGTTATATCCCGCCTCGCCCA']
{'CCC': 8, 'CCT': 8, 'CTC': 7, 'TCC': 5, 'CCG': 2, 'CGC': 7, 'GCT': 3, 'CTA': 4, 'TAC': 2, 'ACG': 1, 'GCA': 2, 'CAG': 2, 'AGC': 2, 'GCG': 3, 'GCC': 3, 'CGT': 2, 'GTA': 2, 'TAT': 6, 'ATC': 2, 'CTG': 1, 'TGC': 2, 'ACC': 1, 'TCT': 2, 'TCG': 2, 'ATA': 2, 'ATT': 1, 'TAG': 1, 'AGT': 1, 'ATG': 1, 'TGT': 1, 'GTG': 1, 'GTT': 1, 'TTA': 1, 'CCA': 1}
{'CCC': 8, 'CCT': 8, 'CTC': 7, 'CGC': 7}


4\. Quell est la complexité de votre algorithme?

<font color='blue'>
reponse : 
    complexiter lineaire:
        O(n*top-log(top))
</font>

5\. Certains motifs sont reverse complémentaires. Implanter des motifs reverse complémentaires dans les séquences déjà générés et en suite chercher ces motifs. Attention: vous devez réutiliser les fonctions développés précédemment, pas besoin de définir des nouvelles fonctions. 

In [21]:
#tme2
def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence.
    entrée : sequence de nucléotides (brin sens)
    sortie : sequence de nucléotides (brin complementaire)
    >>> reversecompl('AACGTGGCA')
    'TGCCACGTT'
    """
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    return "".join( [compl[seq[i]] for i in range(len(seq)-1,-1,-1) ] )

print(motifs)
reverseMotifs=[reversecompl(i) for i in motifs]
print(reverseMotifs)
print(seqs)
for i in reverseMotifs:
    implantMotifs(i, f, motif_implanted_seqs)
print(seqs)
motifsFound = searchMotifs(k, motif_implanted_seqs)
print (motifsFound)
topMotifs = getTopMotifs(motifsFound, 2)
print (topMotifs)

['CCT', 'CGC', 'TAT']
['AGG', 'GCG', 'ATA']
['CCCCTCCCGCTACGCAGCGCCCTCCT', 'CGTATCCCTGCTACCCTCTCGCAGCGCTCTATATT', 'CCTCCTAGTATGTGCGTTATATCCCGCCTCGCCCA']
['CCCCTCCCGCTACGCAGCGCCCTCCT', 'CGTATCCCTGCTACCCTCTCGCAGCGCTCTATATT', 'CCTCCTAGTATGTGCGTTATATCCCGCCTCGCCCA']
{'CGT': 2, 'GTA': 2, 'TAT': 6, 'ATC': 3, 'TCC': 3, 'CCC': 3, 'CCT': 4, 'CTG': 1, 'TGC': 2, 'GCG': 8, 'CGG': 3, 'GGC': 6, 'GCT': 2, 'CTA': 3, 'TAG': 6, 'AGC': 2, 'CGC': 5, 'GCC': 3, 'CTC': 4, 'TCT': 2, 'TCG': 2, 'CGA': 3, 'GAA': 2, 'AAT': 3, 'ATA': 9, 'TAC': 1, 'ACA': 1, 'CAG': 1, 'TAA': 2, 'AGG': 5, 'GGA': 1, 'GAT': 2, 'AGA': 1, 'GAG': 1, 'GGG': 2, 'GGT': 1, 'GCA': 2, 'CAT': 2, 'AGT': 1, 'ATG': 1, 'TGT': 1, 'GTG': 1, 'GTT': 1, 'TTA': 1, 'CCG': 1, 'AAG': 1, 'CCA': 1}
{'ATA': 9, 'GCG': 8}



## Partie B : Recherche de motifs identiques sur vos données

1\. Le fichier "C_glabrata_1000bp_only.fasta" contiens les séquences régulatrices pour tous les gènes de votre organisme. Nous avons pris les 1000bp en amont du codon start. Chercher les 50 motifs de taille 7 les plus fréquents dans les deux brim de votre génome.

In [22]:
k=7
feq=10
top= 50
genome = "Data/C_glabrata_1000bp_only.fasta"


def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    return sequence

sequences = readFasta(genome)

print("Searching motifs...\n")
motifsRes=searchMotifs(k, sequences)
print(list(motifsRes.items())[:10],"...")



print("The most abundant motifs are \n")
topMotifs=getTopMotifs(motifsRes, top)
print(topMotifs)

print("Searching reverse complementary motifs...\n")
reverseCompl=[reversecompl(s) for s in sequences]
reverseComplMotifs=searchMotifs(k, reverseCompl)
print(list(reverseComplMotifs.items())[:10],"...")


print("The most abundant motifs found in reverse complementary are \n")
topReverseComplMotifs=getTopMotifs(reverseComplMotifs, top)
print(topReverseComplMotifs)








Searching motifs...

[('CAAACTG', 515), ('AAACTGG', 562), ('AACTGGC', 303), ('ACTGGCA', 397), ('CTGGCAC', 234), ('TGGCACA', 350), ('GGCACAG', 265), ('GCACAGA', 531), ('CACAGAG', 545), ('ACAGAGA', 640)] ...
The most abundant motifs are 

{'AAAAAAA': 8385, 'TTTTTTT': 7658, 'ATATATA': 3096, 'ATTTTTT': 2970, 'AAAAAAT': 2960, 'TATATAT': 2796, 'AAAAATT': 2469, 'AATTTTT': 2457, 'GAAAAAA': 2417, 'AAAGAAA': 2417, 'AAAATTT': 2308, 'AAATTTT': 2305, 'TATTTTT': 2292, 'TTTTTTC': 2291, 'AAGAAAA': 2243, 'TATATAA': 2198, 'AAAAATA': 2188, 'TTTATTT': 2164, 'TTTCTTT': 2159, 'TGAAAAA': 2152, 'TTATTTT': 2134, 'TTTTATT': 2110, 'AAAAGAA': 2079, 'TTTTCTT': 2052, 'TTTTTCA': 1977, 'CAAAAAA': 1964, 'ATATAAA': 1946, 'AAAATAA': 1940, 'TTCTTTT': 1912, 'ATATATT': 1904, 'AATATAT': 1898, 'AATAAAA': 1889, 'AAATAAA': 1880, 'AGAAAAA': 1864, 'TTTTTAT': 1862, 'CTTTTTT': 1861, 'AAAAAAG': 1853, 'AATAATA': 1824, 'TTTTTTA': 1813, 'TTTTTTG': 1807, 'TTTTTCT': 1804, 'AAATATA': 1797, 'ATATTTT': 1781, 'ATAAAAA': 1780, 'TATATTT': 177

2\. Quel sont les trois motifs de taille 7 les plus frequents? Pensez vous que ces motifs correspondent à de facteur de transcription connus? Justifier votre reponse.

<font color='blue'>
reponse:
   brun sens = {'AAAAAAA': 8385, 'TTTTTTT': 7658, 'ATATATA': 3096}
   brun reverse complementaire = {'TTTTTTT': 8385, 'AAAAAAA': 7658, 'TATATAT'}
  
Ces motifs ne correspondent pas à des facteur de transcription connus.
Les plus connues sont TATAAA, GGGCGG.
</font>

3\. Le motif TGATTCAT correspond au facteur de transcription Gcn4 qui est trés suivant trouvé dans le genome de levures. Est-ce que vous avez trouvé ce motif? Si oui avec quel frequence?
<font color='blue'>
    oui, TGATTCAT freq=
        sens : 0.036
        reverse complementaire : 0.0362
</font>

In [23]:
#TGATTCAT freq=0.036071777343 : 0.036254882812


motifGcn4 = ['TGATTCA']

def searchGivenMotif(motifsTrouve, motifSpecifique):
    """
    Cherche une liste de motifs specifiques dans un dictionaire de motifs trouvés
    entrée motifsTrouve : dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    entrée motifSpecifique: liste de motifs specifiques à chercher
    sortie None : affiche le ranking des motifs especifique et sa frequence d'observation
    >>>searchGivenMotif({'TAA': 4, 'AAG': 3, 'AGT': 1}, ['AAG'])
    2 - AGG - 3
    """
    l=[]
    for k,v in sorted(motifsTrouve.items(),key=lambda x: x[1]):  
        l=[(k,v)]+l
    for m in motifSpecifique:
        if m in motifsTrouve:
            print(l.index((m,motifsTrouve[m]))+1,'-',m,'-',motifsTrouve[m])
    return None

motifsData=motifsRes
motifsRevComData=reverseComplMotifs
searchGivenMotif(motifsData, motifGcn4)    
print ("reverse")
searchGivenMotif(motifsRevComData, motifGcn4)   



1819 - TGATTCA - 591
reverse
1789 - TGATTCA - 594


4\. Les motifs peu complexes (avec par exemple 5, 6 ou 7 fois la même lettre) sont courants dans les génomes, ils n'ont généralement pas de signification biologique. Dans le context de ce projet, vous pouvez eliminer aussi les motifis ayant deux lettre repetés, comme par exemple AGAGAGA. Faites une fonction pour éliminer les motifs peu complexes, _i. e._ qui ont au moins $m$ fois la même lettre ou qui ont deux lettre consecutive répétés. Dans quel position vous trouver Gcn4 apres enlever les motifs peu complexes?</font>. 

In [15]:
# m fois la mme lettre
reLow=5
def removeLowComplexe(motifs):
    """
    Eleve les motifs peu complexe 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie validMotif: dictionnaire de motifs filtré, clé=motif, valeur = fréquence d'observation
    """
    validMotif = {}
    for mo in motifs:
        k=0
        cpt=0
        for i in range(1,len (mo)-reLow+2):
            k=1
            cpt=1
            while( cpt/2<=2 and k<=reLow  ):
                if i+k-1<len(mo) and mo[i+k-2]==mo[i+k-1]:
                    k+=1
                elif i+cpt<len(mo) and mo[i+cpt-2]==mo[i+cpt]:
                    cpt+=1
                else:
                    break  
            if k>=reLow or cpt/2>=2:#repetition de lettres superieures a m
                break #motif suivant
        if k<reLow and cpt/2<2:#si aucune rep
            validMotif[mo]=motifs[mo] #add
            
    return validMotif


motifsFiltreData = removeLowComplexe(motifsData)
motifsFiltreReverse = removeLowComplexe(motifsRevComData)
searchGivenMotif(motifsFiltreData, motifGcn4)    
print ("reverse")
searchGivenMotif(motifsFiltreReverse, motifGcn4)


1627 - TGATTCA - 591
reverse
1597 - TGATTCA - 594


5\. Le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer les fonctions precedents pour chercher les 3 motifs les plus fréquents dans les deux brim. Il faut bien evidement enlever les motifs peu complexe.

In [16]:
k=8
feq=5
top= 30
reLow = 5

genome = "Data/ChipSeq/Sequence_by_Peaks_8.fasta"

sequence   = readFasta(genome)


print("Searching fwd motifs...\n")
motifsRes=searchMotifs(k, sequence)
print(list(motifsRes.items())[:10],"...")

print("The most abundant motifs are \n")
topMotifs=getTopMotifs(motifsRes, top)
print(topMotifs)

print("The most abundant reverse compl motifs are \n")
reverseCompl=[reversecompl(s) for s in sequence]
reverseComplMotifs=searchMotifs(k, reverseCompl)
topReverseComplMotifs=getTopMotifs(reverseComplMotifs, top)
print(topReverseComplMotifs)

print("The most abundant no low-complexe motifs are \n")
motifsFiltreData=removeLowComplexe(topMotifs)
print(list(motifsFiltreData.items())[:top],"...")

print("The most abundant no low-complexe motifs in reverse compl are \n")
motifsFiltreReverse=removeLowComplexe(topReverseComplMotifs)
print(list(motifsFiltreReverse.items())[:top],"...")


#sens ('TTATTATT', 17), ('ATTATTAT', 16), ('TTATTTAT', 16)
#compl ('AATAATAA', 17), ('ATAATAAT', 16), ('ATAAATAA', 16)

Searching fwd motifs...

[('AAATTATT', 8), ('AATTATTG', 6), ('ATTATTGG', 7), ('TTATTGGT', 3), ('TATTGGTT', 2), ('ATTGGTTC', 3), ('TTGGTTCA', 1), ('TGGTTCAA', 4), ('GGTTCAAG', 2), ('GTTCAAGA', 2)] ...
The most abundant motifs are 

{'ATATATAT': 46, 'TATATATA': 45, 'AAATATAT': 20, 'TTTTTTTT': 19, 'AAAAAAAA': 18, 'TTATTATT': 17, 'ATATAATA': 17, 'ATTATTAT': 16, 'TTATTTAT': 16, 'TTTATTAT': 16, 'TATTAATT': 15, 'TAATAATA': 15, 'ATTTTTTT': 15, 'TCCACGGA': 15, 'TTTATTTA': 14, 'ATTTATTA': 14, 'TAATTATT': 14, 'TTATTTTT': 14, 'ATAAATAT': 14, 'TAAATATA': 14, 'AATATATT': 14, 'ATATATTA': 14, 'ATTAATTA': 14, 'ATATATAA': 14, 'TATAATAT': 14, 'TATATATT': 14, 'AATAATAA': 14, 'CCACGGAA': 14, 'CACACACA': 14, 'CCAAAACC': 14}
The most abundant reverse compl motifs are 

{'ATATATAT': 46, 'TATATATA': 45, 'ATATATTT': 20, 'AAAAAAAA': 19, 'TTTTTTTT': 18, 'AATAATAA': 17, 'TATTATAT': 17, 'ATAATAAT': 16, 'ATAAATAA': 16, 'ATAATAAA': 16, 'AATTAATA': 15, 'TATTATTA': 15, 'AAAAAAAT': 15, 'TCCGTGGA': 15, 'TAAATAAA': 14, 'T

6\. Ulitser la base YEASTRACT : http://www.yeastract.com/formsearchbydnamotif.php pour chercher les motifs. 
Avez vous une indication pour le facteur de transcription impliqué ?


In [ ]:
Pdr1 : TCCG(C/T)GGA